# AU demo using shapes_6dof dataset on Ultra96

This jupyter notebook runs the hardware demo of REMOT on shape_6dof dataset using 3 implementation: 
- FIFO-ONLY
- HASH-AMAP 
- Full-AMAP

Run through the notebook, the results of bbox, idx and video will be stored on the ./result folder

In [7]:
from sklearn.cluster import DBSCAN, AgglomerativeClustering
from scipy import io
from scipy.spatial.distance import squareform, directed_hausdorff
from itertools import combinations
import numpy as np
from pynq import Overlay 
from pynq import allocate
from pynq import Xlnk
import numpy as np
import time
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import math
import cv2
from tqdm import tqdm
from matplotlib import pyplot as plt
import argparse
import os
import csv
import yaml
from au_functions import *
from au_hardware_hash import Au_hash
from au_hardware_full import Au_full
from au_hardware_fifo_only import Au_fifo



# AU controller 

In [18]:

class Controller():
    def __init__(self, args):
        self.input = args.input
        self.bitfile = args.bitfile
        self.Init(self.input)
        self.tkBoxes = []
        self.tkIDs = []

        self.auFifo = args.auFifo
        self.auNum = args.auNum
        self.dAdd = args.dAdd
        self.folder = args.outfolder
        self.name = args.name

        self.tDel = args.tDel * self.tFrame
        self.areaDel = args.areaDel
        self.tLive = args.tLive * self.tFrame
        self.areaLive = args.areaLive
        self.numLive = args.numLive

        self.t = self.tFrame

        self.split = args.split
        self.epsDiv = args.epsDiv
        self.minptsDiv = 1

        self.merge = args.merge
        self.iomMer = args.iomMer
        self.dsMer = args.dsMer
        self.minptsMer = 1
        self.epsMer = 1

        if "inbound" in self.input:
            self.bdspawn1 = 40
            self.bdspawn2 = 40
        else:
            self.bdspawn1 = -1
            self.bdspawn2 = -1

        if "outbound" in self.input:
            self.bdkill = 20
        else:
            self.bdkill = -1

        self.globalID = -1
        if 'hash' in args.bitfile:
            self.AUs = Au_hash(Height=self.ly, Width=self.lx, bitfile=args.bitfile, au_number=args.auNum, fifo_depth=args.auFifo, dAdd=self.dAdd)
        elif 'full' in args.bitfile:
            self.AUs = Au_full(Height=260, Width=342, bitfile=args.bitfile, au_number=args.auNum, fifo_depth=args.auFifo, dAdd=self.dAdd)
        elif 'fifo' in args.bitfile:
            self.AUs = Au_fifo(Height=self.ly, Width=self.lx, bitfile=args.bitfile, au_number=args.auNum, fifo_depth=args.auFifo)
        else:
            raise ValueError("Not matching bitfile overlay")

        self.frame_count = 0
        x = self.events[:, 0]
        y = self.events[:, 1]
        p = self.events[:, 2]
        self.t = self.events[:, 3]
        self.total_time = self.t[-1]
        self.events = np.vstack([x, y, self.t, p]).T
            
            
    def Init(self, file):
        if "bound" in self.input:
            self.tFrame = 40000
        else:
            self.tFrame = 44065

        self.cmap = io.loadmat('cmap.mat')['cmap'] * 255
        self.events = io.loadmat(file)['events']
        self.lx, self.ly = self.events[:, :2].max(0) + 1
        self.pFrame = -1
        self.nFrame = self.events[:, 4][-1] + 1
        self.frame0 = np.zeros((self.ly, self.lx, 3), 'uint8')
        self.iFrame = self.frame0.copy()
        self.frames = np.tile(self.frame0, (self.nFrame, 1, 1, 1))

    def update_box(self, ts):
        live_au_list = np.where(self.AUs.status_reg == 0)[0]
        for i in live_au_list:
            auEvents = self.AUs.au_event_fifo[i]
            idxFade = np.argwhere(auEvents[:, 2] < ts - self.tFrame).flatten()

            idxFade = np.argwhere(auEvents[:, 2] < auEvents[-1, 2] - self.tFrame).flatten()
            if idxFade.size != 0:
                auEvents = np.delete(auEvents, idxFade, axis=0)
            self.AUs.au_event_fifo[i] = auEvents
            self.AUs.auBox[i] = bbox(auEvents[:, 0], auEvents[:, 1])
                
        
    def Split(self):
        idxDel = []
        if (np.sum(self.AUs.status_reg) == 0): 
            return 
        
        for j in np.where(self.AUs.status_reg != 1)[0]:
            auEvents = self.AUs.au_event_fifo[j] 
            if self.split == 'DBSCAN':
                idxGroup = DBSCAN(eps=self.epsDiv, min_samples=self.minptsDiv).fit_predict(
                    auEvents[:, :2])
                idxGroup[idxGroup < 0] = 0
            else:
                if au.auEvents.shape[0] <= 1:
                    continue
                clustering = AgglomerativeClustering(
                    linkage='average', affinity='euclidean').fit(auEvents[:, :2])
                idxGroup = clustering.labels_
                idxGroup[idxGroup < 0] = 0
                if max(directed_hausdorff(auEvents[idxGroup == 0, :2], auEvents[idxGroup == 1, :2])[0],
                       directed_hausdorff(auEvents[idxGroup == 1, :2], auEvents[idxGroup == 0, :2])[0]) < self.dsMer:
                    continue

            if max(idxGroup) <= 0: 
                continue
            else:
                idxDel.append(j)

            idxTk = np.argmax([sum(idxGroup == idx)
                              for idx in np.unique(idxGroup)])

            for k in range(max(idxGroup) + 1):
                next_empty = np.where(self.AUs.status_reg == 1)[0]
                if next_empty.shape[0] ==0:
                    return
                else:
                    next_empty = next_empty[0]
                    
                    idxEvents = np.argwhere(idxGroup == k).flatten()
                    event_collect = auEvents[idxEvents]
                    self.AUs.auBox[next_empty] = bbox(event_collect[:, 0], event_collect[:, 1]) 
                    
                    self.AUs.write_au(event=event_collect, number=next_empty)
                    if k == idxTk:
                        self.AUs.auNumber[next_empty] = self.AUs.auNumber[j] 
                    else:
                        self.AUs.auNumber[next_empty] = [0, min(event_collect[:, 2])]

        if len(idxDel) > 0:
            for idx in (idxDel):
                self.AUs.kill_au(idx)
                
    def Merge(self):
        live_au_list = np.where(self.AUs.status_reg == 0)[0]
        if live_au_list.shape[0] <= 1: ## if state reg only has one 0
            return
        
        idxnk = list(combinations(live_au_list, 2)) 
        idxGroup = clusterAu(np.array(self.AUs.auBox)[live_au_list], self.iomMer)

        for j in range(max(idxGroup) + 1):
            idxAU = np.argwhere(idxGroup == j).flatten()
            idxAU = live_au_list[idxAU]
            idxDel = idxAU
            if idxAU.size < 2:
                continue
            
            write_au_idx = np.min(idxAU)
            print("merging{} to {}".format(idxAU, write_au_idx) )
            events = np.concatenate(
                [self.AUs.au_event_fifo[idx] for idx in idxAU], axis=0) 
            events = np.unique(events, axis=0)
            events = events[np.argsort(events[:, 2])]
        
            if any([self.AUs.auNumber[idx][0] > 0 for idx in idxAU]):
                idxAU = idxAU[[self.AUs.auNumber[idx][0] > 0 for idx in idxAU]]
            idxNum = idxAU[np.argmin([self.AUs.auNumber[idx][0] for idx in idxAU])]
            
            self.AUs.write_au(event=events, number=write_au_idx)
            self.AUs.auBox[write_au_idx] = bbox(events[:, 0], events[:, 1])
            self.AUs.auNumber[write_au_idx] = self.AUs.auNumber[idxNum]
            
            for idx in idxDel:
                if idx == write_au_idx:
                    continue
                self.AUs.kill_au(idx)

    def Kill(self, ts):
        live_au_list = np.where(self.AUs.status_reg==0)[0]
        idxDel = []
        
        for idx in live_au_list:
            flag1 = ts - np.max(self.AUs.au_event_fifo[idx][:, 2]) > self.tDel
            flag2 = bbArea(self.AUs.auBox[idx]) < self.areaDel
            flag3 = (self.AUs.auBox[idx][1] + self.AUs.auBox[idx][3]) / 2 < self.bdkill
            if flag1 or flag2 or flag3:
                idxDel.append(idx)
                                 
                              
        if len(idxDel) > 0:
            print("killing", idxDel)
            for idx in idxDel:
                self.AUs.kill_au(idx)  

    def UpdateID(self, ts):
        if 'fifo' in self.bitfile:
            self.AUs.write_all_au()
            
        live_au_list = np.where(self.AUs.status_reg==0)[0]
        for idx in live_au_list:
            if not self.AUs.auNumber[idx][0] and \
            ts - self.AUs.auNumber[idx][1] > self.tLive and \
            bbArea(self.AUs.auBox[idx]) > self.areaLive and \
            self.AUs.au_event_fifo[idx].shape[0] > self.numLive and \
            self.AUs.auBox[idx][2] / 2 > self.bdspawn1 and \
            self.AUs.auBox[idx][3] / 2 > self.bdspawn2:
                self.globalID += 1
                self.AUs.auNumber[idx][0] = self.globalID

    def Animation(self, events, ts):
        self.iFrame[:] = 0
        y = events[:, 1]
        x = events[:, 0]
        self.iFrame[y, x] = [255, 255, 255]
        
        idxVis = []
        boxes = []
        IDs = []

        live_au_list = np.where(self.AUs.status_reg == 0)[0]
        cmap_idx = np.array([self.AUs.auNumber[i][0] % 7 for i in live_au_list], 'int32')
        auColors = np.zeros([self.auNum, 3])
        auColors[live_au_list] = self.cmap[cmap_idx]

        for j in live_au_list:
            if self.AUs.auNumber[j][0] > 0:
                idxEvt = self.AUs.au_event_fifo[j][:, 2] >= ts - self.tFrame
                if any(idxEvt):
                    idxVis.append(j)
                    one_frame_events = self.AUs.au_event_fifo[j][idxEvt, :]
                    boxes.append(
                        bbox(one_frame_events[:, 0], one_frame_events[:, 1]))
                    IDs.append(self.AUs.auNumber[j][0])

        self.tkBoxes.append(boxes)
        self.tkIDs.append(IDs)

        if len(idxVis) > 0:
            for j, k in enumerate(idxVis):
                self.iFrame = cv2.rectangle(
                    self.iFrame, (boxes[j][0], boxes[j][1]), (boxes[j][2], boxes[j][3]), auColors[k].tolist(), 1)

        if len(idxVis) > 0:
            for j, k in enumerate(idxVis):
                self.iFrame = cv2.putText(self.iFrame, '{}'.format(
                    IDs[j]), (boxes[j][0], boxes[j][1]), cv2.FONT_HERSHEY_PLAIN, 1., auColors[k].tolist(), 1)

        self.frames[self.frame_count] = self.iFrame


    def SaveResults(self):
        if not os.path.exists(self.folder):
            os.mkdir(self.folder)

        tkbox_dir = os.path.join(self.folder, self.name + 'tkBoxes.mat')
        tkid_dir = os.path.join(self.folder, self.name + 'tkIDs.mat')
        gt_dir = self.input.replace('.mat', 'GT.mat')
        video_dir = os.path.join(self.folder, self.name + 'output.avi')

        for boxes in self.tkBoxes:
            if len(boxes) == 0:
                continue
            for box in boxes:
                box[2] = box[2] - box[0]
                box[3] = box[3] - box[1]

        self.tkBoxes = np.array(self.tkBoxes, dtype='object')
        self.tkIDs = np.array(self.tkIDs, dtype='object')
        io.savemat(tkbox_dir, {'tkBoxes': self.tkBoxes})
        io.savemat(tkid_dir, {'tkIDs': self.tkIDs})

        fourcc = cv2.VideoWriter_fourcc(*'XVID')
        videoWriter = cv2.VideoWriter(
            video_dir, fourcc, 5, (self.lx, self.ly), True)
        for i in range(self.frame_count):
            videoWriter.write(self.frames[i])
        videoWriter.release() 
        print("Saving results in ", tkbox_dir, tkid_dir, video_dir)
                
    def Process(self, save_result=True):
        
        for i in range(self.total_time // (1 * self.tFrame)):
            print("Processing {}---------------------------------------".format(self.frame_count))
            idxs = np.where((self.t > self.frame_count * self.tFrame) & (self.t <= (self.frame_count + 1) * 1 * self.tFrame))[0]
            events = self.events[idxs]
            ts = events[-1, 2]
            self.AUs.stream_in_events(events)
            self.AUs.dump_all_au()
            self.update_box(ts)
            
            self.Split()
            self.Merge()
            self.Kill(ts)
            self.UpdateID(ts)    
            if save_result:
                self.Animation(events, ts)
            self.frame_count += 1

        
        if save_result:
            self.SaveResults()


    
            
 

## FIFO-ONLY  

In [19]:
config_dir = "./config/shape_6dof_fifo.yml"

with open(config_dir, "r") as file:
    config = yaml.safe_load(file)

config = ARGS(config)
controller = Controller(config)
controller.Process()

Processing 0---------------------------------------
processing 226 event using:0.00031185150146484375
killing [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
Processing 1---------------------------------------
processing 997 event using:0.00047278404235839844
merging[0 5] to 0
merging[ 1  2 11] to 1
merging[3 7] to 3
killing [1, 4, 9]
Processing 2---------------------------------------
processing 1084 event using:0.00042128562927246094
merging[0 4 8] to 0
merging[2 6] to 2
merging[3 5] to 3
killing [7, 11]
Processing 3---------------------------------------
processing 1352 event using:0.00045108795166015625
merging[1 5] to 1
merging[ 4  7 10] to 4
killing [11]
Processing 4---------------------------------------
processing 674 event using:0.0004277229309082031
merging[ 4 11] to 4
merging[6 9] to 6
killing [7, 8, 10]
Processing 5---------------------------------------
processing 196 event using:0.00041413307189941406
merging[0 9] to 0
merging[1 8] to 1
merging[ 4 10] to 4
killing [5, 7, 11]
Proce

processing 4630 event using:0.0011119842529296875
merging[ 2  3  9 10 11] to 2
killing [6]
Processing 55---------------------------------------
processing 4399 event using:0.0010700225830078125
merging[ 2  3  9 10 11] to 2
Processing 56---------------------------------------
processing 4127 event using:0.0010116100311279297
merging[ 2  3  9 10] to 2
merging[ 4 11] to 4
merging[6 8] to 6
Processing 57---------------------------------------
processing 3340 event using:0.0008616447448730469
merging[ 1 11] to 1
merging[ 2  3  8  9 10] to 2
Processing 58---------------------------------------
processing 1968 event using:0.0005745887756347656
merging[2 3 8] to 2
merging[10 11] to 10
killing [9, 10]
Processing 59---------------------------------------
processing 991 event using:0.00043010711669921875
merging[0 8] to 0
merging[ 2  3  9 10] to 2
merging[ 7 11] to 7
Processing 60---------------------------------------
processing 329 event using:0.00042724609375
merging[1 3] to 1
merging[ 4  9 10

merging[ 3 10] to 3
merging[4 8] to 4
killing [0, 9]
Processing 109---------------------------------------
processing 5816 event using:0.0013573169708251953
merging[ 0  9 11] to 0
merging[5 6] to 5
merging[7 8] to 7
killing [2, 10]
Processing 110---------------------------------------
processing 3492 event using:0.0008869171142578125
merging[ 0  7 10] to 0
merging[ 5 11] to 5
killing [2, 6, 8, 9]
Processing 111---------------------------------------
processing 5773 event using:0.0013501644134521484
merging[ 0 11] to 0
merging[1 8] to 1
merging[ 5  7  9 10] to 5
killing [2, 6]
Processing 112---------------------------------------
processing 5675 event using:0.0013244152069091797
merging[0 6] to 0
merging[2 4] to 2
killing [7, 8, 9, 10, 11]
Processing 113---------------------------------------
processing 4268 event using:0.0010440349578857422
merging[1 9] to 1
merging[2 8] to 2
merging[ 4  5 11] to 4
killing [6, 7, 10]
Processing 114---------------------------------------
processing 5133

merging[ 1 11] to 1
merging[4 8] to 4
killing [5]
Processing 164---------------------------------------
processing 13118 event using:0.002830028533935547
merging[0 5] to 0
merging[ 4 11] to 4
merging[ 7  8 10] to 7
Processing 165---------------------------------------
processing 9528 event using:0.002100229263305664
merging[0 8] to 0
merging[1 5] to 1
merging[ 3 10] to 3
killing [11]
Processing 166---------------------------------------
processing 7868 event using:0.0017590522766113281
merging[ 1 10] to 1
merging[2 5] to 2
merging[4 8] to 4
merging[ 6 11] to 6
Processing 167---------------------------------------
processing 7449 event using:0.0016803741455078125
killing [5, 8, 10, 11]
Processing 168---------------------------------------
processing 2840 event using:0.0007529258728027344
merging[ 1 11] to 1
merging[6 8] to 6
killing [5, 9, 10]
Processing 169---------------------------------------
processing 4080 event using:0.0010101795196533203
merging[ 3  5  9 11] to 3
merging[ 6 10] 

Processing 220---------------------------------------
processing 4052 event using:0.001051187515258789
merging[ 1 10] to 1
merging[ 4 11] to 4
merging[7 9] to 7
Processing 221---------------------------------------
processing 4306 event using:0.0010488033294677734
merging[0 9] to 0
merging[ 7 10] to 7
killing [11]
Processing 222---------------------------------------
processing 4414 event using:0.0010788440704345703
merging[ 6  9 10] to 6
Processing 223---------------------------------------
processing 4121 event using:0.0010099411010742188
merging[ 0 10] to 0
merging[ 6 11] to 6
killing [9]
Processing 224---------------------------------------
processing 4955 event using:0.0011794567108154297
merging[ 9 11] to 9
killing [9, 10]
Processing 225---------------------------------------
processing 3411 event using:0.0008721351623535156
merging[ 1 10 11] to 1
merging[3 9] to 3
Processing 226---------------------------------------
processing 3417 event using:0.0008611679077148438
merging[ 1 1

## Full AMAP

In [20]:
config_dir = "./config/shape_6dof_full.yml"

with open(config_dir, "r") as file:
    config = yaml.safe_load(file)

config = ARGS(config)

controller = Controller(config)
controller.Process()

Processing 0---------------------------------------
processing 226 event using:0.0003066062927246094
killing [1, 2, 3]
Processing 1---------------------------------------
processing 997 event using:0.0006299018859863281
Processing 2---------------------------------------
processing 1084 event using:0.0006709098815917969
Processing 3---------------------------------------
processing 1352 event using:0.0007839202880859375
Processing 4---------------------------------------
processing 674 event using:0.0004868507385253906
Processing 5---------------------------------------
processing 196 event using:0.0003809928894042969
Processing 6---------------------------------------
processing 865 event using:0.0005650520324707031
Processing 7---------------------------------------
processing 806 event using:0.0005459785461425781
Processing 8---------------------------------------
processing 1624 event using:0.0009245872497558594
Processing 9---------------------------------------
processing 1906 ev

processing 7235 event using:0.003498077392578125
Processing 81---------------------------------------
processing 6857 event using:0.0033266544342041016
Processing 82---------------------------------------
processing 6454 event using:0.0031409263610839844
Processing 83---------------------------------------
processing 6620 event using:0.003220796585083008
Processing 84---------------------------------------
processing 5944 event using:0.0029022693634033203
Processing 85---------------------------------------
processing 4431 event using:0.0022068023681640625
Processing 86---------------------------------------
processing 4253 event using:0.0021300315856933594
Processing 87---------------------------------------
processing 4838 event using:0.002424955368041992
Processing 88---------------------------------------
processing 5068 event using:0.0025033950805664062
Processing 89---------------------------------------
processing 5029 event using:0.0024857521057128906
Processing 90-------------

processing 6774 event using:0.003275156021118164
Processing 160---------------------------------------
processing 7128 event using:0.0034379959106445312
Processing 161---------------------------------------
processing 8695 event using:0.00415802001953125
Processing 162---------------------------------------
processing 10405 event using:0.004938364028930664
Processing 163---------------------------------------
processing 9944 event using:0.004731655120849609
Processing 164---------------------------------------
processing 13118 event using:0.0061953067779541016
Processing 165---------------------------------------
processing 9528 event using:0.004544258117675781
Processing 166---------------------------------------
processing 7868 event using:0.0037839412689208984
Processing 167---------------------------------------
processing 7449 event using:0.0035965442657470703
Processing 168---------------------------------------
processing 2840 event using:0.0014781951904296875
Processing 169----

## HASH-AMAP

In [21]:
config_dir = "./config/shape_6dof_hash.yml"

with open(config_dir, "r") as file:
    config = yaml.safe_load(file)

config = ARGS(config)
    
controller = Controller(config)
controller.Process()

Processing 0---------------------------------------
processing 226 event using:0.0002810955047607422
merging[0 7] to 0
killing [1, 2, 3, 4, 6, 8, 9, 11]
Processing 1---------------------------------------
processing 997 event using:0.0006613731384277344
merging[1 4 7] to 1
merging[ 2  8 11] to 2
merging[3 6] to 3
Processing 2---------------------------------------
processing 1084 event using:0.0007054805755615234
merging[ 2  4  8 11] to 2
merging[ 3 10] to 3
Processing 3---------------------------------------
processing 1352 event using:0.0008332729339599609
merging[ 2 10] to 2
merging[3 5] to 3
merging[ 4  6 11] to 4
merging[7 8] to 7
Processing 4---------------------------------------
processing 674 event using:0.0005002021789550781
merging[2 6] to 2
merging[ 3 11] to 3
merging[5 9] to 5
killing [8, 10]
Processing 5---------------------------------------
processing 196 event using:0.00060272216796875
merging[ 2 10 11] to 2
merging[3 9] to 3
killing [6, 8]
Processing 6----------------

merging[ 2  3  9 11] to 2
merging[ 8 10] to 8
killing [4]
Processing 55---------------------------------------
processing 4399 event using:0.002344369888305664
merging[ 2  3  4  9 10] to 2
killing [11]
Processing 56---------------------------------------
processing 4127 event using:0.0021944046020507812
merging[2 3 4] to 2
merging[5 9] to 5
merging[ 8 11] to 8
killing [10]
Processing 57---------------------------------------
processing 3340 event using:0.0019016265869140625
merging[ 1 10 11] to 1
merging[2 3 4 9] to 2
Processing 58---------------------------------------
processing 1968 event using:0.0011382102966308594
merging[ 2  3 10] to 2
merging[4 8] to 4
merging[ 5 11] to 5
killing [9]
Processing 59---------------------------------------
processing 991 event using:0.0006611347198486328
merging[ 1 10 11] to 1
merging[2 3 9] to 2
killing [8]
Processing 60---------------------------------------
processing 329 event using:0.00039768218994140625
merging[ 0 11] to 0
merging[ 1 10] to 1


processing 5816 event using:0.0030405521392822266
merging[ 3 11] to 3
merging[4 9] to 4
merging[ 7 10] to 7
killing [1, 2, 8]
Processing 110---------------------------------------
processing 3492 event using:0.0018913745880126953
merging[0 9] to 0
merging[2 6] to 2
merging[ 5 10] to 5
merging[ 8 11] to 8
killing [1, 8]
Processing 111---------------------------------------
processing 5773 event using:0.0030164718627929688
merging[ 4  8 10] to 4
merging[ 6 11] to 6
killing [1, 9]
Processing 112---------------------------------------
processing 5675 event using:0.0029828548431396484
merging[1 2] to 1
merging[ 4  6 11] to 4
merging[ 5 10] to 5
killing [8, 9]
Processing 113---------------------------------------
processing 4268 event using:0.002263784408569336
merging[ 1 11] to 1
merging[2 4] to 2
merging[5 8] to 5
killing [6, 9, 10]
Processing 114---------------------------------------
processing 5133 event using:0.002694368362426758
merging[ 2  6  9 10] to 2
merging[ 7 11] to 7
killing [4

processing 7868 event using:0.004037618637084961
merging[0 8] to 0
merging[4 5] to 4
Processing 167---------------------------------------
processing 7449 event using:0.0038688182830810547
killing [5, 8]
Processing 168---------------------------------------
processing 2840 event using:0.0015683174133300781
merging[ 6 11] to 6
killing [8]
Processing 169---------------------------------------
processing 4080 event using:0.002165555953979492
merging[3 8] to 3
killing [5, 6, 11]
Processing 170---------------------------------------
processing 2772 event using:0.0015268325805664062
merging[ 2 11] to 2
merging[5 8] to 5
killing [6]
Processing 171---------------------------------------
processing 3629 event using:0.0020041465759277344
merging[ 1 11] to 1
merging[2 6] to 2
merging[3 8] to 3
Processing 172---------------------------------------
processing 2798 event using:0.0015463829040527344
merging[2 6 8] to 2
merging[ 3 11] to 3
Processing 173---------------------------------------
processi

processing 3816 event using:0.002040386199951172
merging[2 9] to 2
killing [10]
Processing 228---------------------------------------
processing 2545 event using:0.0014231204986572266
merging[ 0 10] to 0
merging[2 9] to 2
killing [11]
Saving results in  result/shapes_hash_tkBoxes.mat result/shapes_hash_tkIDs.mat result/shapes_hash_output.avi
